In [2]:
import pandas as pd
import os

In [2]:
import gzip
import json
import pandas as pd

# JSON.gz 파일을 읽어 CSV로 변환하는 함수
def json_gz_to_csv(json_gz_file, csv_file):
    # gzip 파일 열기
    with gzip.open(json_gz_file, 'rt', encoding='utf-8') as f:
        # JSON 데이터를 파이썬 객체로 로드
        data = json.loads(f.read())
        
        # 데이터가 리스트 형태라면, 데이터프레임으로 변환
        df = pd.DataFrame(data)
        
        # CSV 파일로 저장
        df.to_csv(csv_file, index=False)

json_gz_file = 'reviews_Clothing_Shoes_and_Jewelry_5.json.gz'  # 입력 파일 경로
csv_file = 'Clothing_Shoes_and_Jewelry.csv'  # 출력 파일 경로
json_gz_to_csv(json_gz_file, csv_file)

JSONDecodeError: Extra data: line 2 column 1 (char 432)

### code for preprocessing

In [29]:
path = os.getcwd()

In [34]:
def string_to_numeric_list(s):
    return [int(num) for num in s.split()]

In [75]:
def make_to_sequence(user_id:str, type_id:str, path:str, dict:int):
    df = pd.read_csv(path)

    group_data = df.copy()

    #data의 ID를 mapping 해야하는 경우
    if dict:

        #mapping
        type_id_mapping = {type:idx+2 for idx,type in enumerate(group_data[type_id].unique())}
        user_id_mapping = {name_dict:idx for idx,name_dict in enumerate(group_data[user_id].unique())}
        
        group_data['type_id'] = group_data[type_id].map(type_id_mapping)
        group_data['user_id'] = group_data[user_id].map(user_id_mapping)

    else:
        group_data['type_id']= group_data[type_id]
        group_data['user_id'] = group_data[user_id]
        group_data.drop(columns=[user_id, type_id])

    #TO sequence
    grouped_df = group_data.groupby('user_id').agg({'type_id': list}).reset_index()
    grouped_df['sequence'] = grouped_df['type_id'].apply(lambda x: ' '.join(map(str, x)))

    result_df = grouped_df[['user_id', 'sequence']]
    result_df['sequence_numeric'] = result_df['sequence'].apply(string_to_numeric_list)
    result_df = result_df.drop(columns = ['sequence'])
    

    return result_df, group_data
    

    

### ML-1m 

In [ ]:
column_names = ['UserID', 'MovieID', 'Rating', 'Timestamp']
df = pd.read_csv('/home/sb/data/ml-20m.inter', header=None, names =column_names, engine= 'python' )
df

In [46]:
column_names = ['UserID', 'MovieID', 'Rating', 'Timestamp']

df = pd.read_csv(f'{path}/ml-1m/ratings.dat', sep='::', header=None, names =column_names, engine= 'python')
df.to_csv(f'{path}/ml-1m/ratings_original.csv')

In [32]:
df

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [45]:
df.to_csv(f'{path}/m1-1m.csv')

number of unique MovieID

In [43]:
df['MovieID'].nunique()

3706

In [33]:
ml_sample = df.copy()

In [40]:
ml_sample_ = make_to_sequence('UserID','MovieID',f'{path}/ml-1m/ratings_original.csv',0)
ml_sample_

,user_id,sequence_numeric
0,1,"[1193, 661, 914, 3408, 2355, 1197, 1287, 2804,..."
1,2,"[1357, 3068, 1537, 647, 2194, 648, 2268, 2628,..."
2,3,"[3421, 1641, 648, 1394, 3534, 104, 2735, 1210,..."
3,4,"[3468, 1210, 2951, 1214, 1036, 260, 2028, 480,..."
4,5,"[2987, 2333, 1175, 39, 288, 2337, 1535, 1392, ..."
...,...,...
6035,6036,"[571, 574, 2053, 2054, 2058, 588, 589, 4, 3005..."
6036,6037,"[589, 3006, 1407, 2064, 2065, 593, 3015, 903, ..."
6037,6038,"[1419, 920, 3088, 232, 1136, 1148, 1183, 2146,..."
6038,6039,"[588, 2067, 1416, 3022, 3028, 2080, 2083, 2087..."


code for statistics

In [41]:
ml_sample_['sequence_length'] = ml_sample_['sequence_numeric'].apply(len)
ml_sample_['sequence_length'].describe()

count    6040.000000
mean      165.597517
std       192.747029
min        20.000000
25%        44.000000
50%        96.000000
75%       208.000000
max      2314.000000
Name: sequence_length, dtype: float64

In [ ]:
column_names = ['MovieID','Title','Genres']
df = pd.read_csv(f'{path}/ml-1m/movies.dat', sep='::', header=None, names =column_names, engine= 'python',encoding='ISO-8859-1')

df.to_csv(f'{path}/data/ml-1m/movies.csv')

### Amazon Beauty

In [3]:
column_names = ['asin', 'reviewerID','overall','unixReviewTime']

df = pd.read_csv('/Users/sb/Desktop/project/preference_rec/data/ratings_Beauty.csv', sep=',',header=None, names=column_names, engine= 'python')
df.to_csv('/Users/sb/Desktop/project/preference_rec/data/ratings_Beauty.csv')

In [161]:
df = df.drop(df.index[0])

In [162]:
df

,item,user,rating,timestamp
1.0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
2.0,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
3.0,A1Z513UWSAAO0F,0558925278,5.0,1404691200
4.0,A1WMRR494NWEWV,0733001998,4.0,1382572800
5.0,A3IAAVS479H7M7,0737104473,1.0,1274227200
...,...,...,...,...
2023066.0,A3DEHKPFANB8VA,B00LORWRJA,5.0,1405296000
2023067.0,A3DEHKPFANB8VA,B00LOS7MEE,5.0,1405296000
2023068.0,AG9TJLJUN5OM3,B00LP2YB8E,5.0,1405382400
2023069.0,AYBIB14QOI9PC,B00LPVG6V0,5.0,1405555200


drop duplicates for same user and item

In [163]:
df = df.drop_duplicates(subset=['item', 'user'])
df

,item,user,rating,timestamp
1.0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
2.0,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
3.0,A1Z513UWSAAO0F,0558925278,5.0,1404691200
4.0,A1WMRR494NWEWV,0733001998,4.0,1382572800
5.0,A3IAAVS479H7M7,0737104473,1.0,1274227200
...,...,...,...,...
2023066.0,A3DEHKPFANB8VA,B00LORWRJA,5.0,1405296000
2023067.0,A3DEHKPFANB8VA,B00LOS7MEE,5.0,1405296000
2023068.0,AG9TJLJUN5OM3,B00LP2YB8E,5.0,1405382400
2023069.0,AYBIB14QOI9PC,B00LPVG6V0,5.0,1405555200


In [158]:
type_id_mapping = {type:idx+2 for idx,type in enumerate(df['item'].unique())}
user_id_mapping = {name_dict:idx for idx,name_dict in enumerate(df['user'].unique())}

In [159]:
df['type_id'] = df['item'].map(type_id_mapping)
df['user_id'] = df['user'].map(user_id_mapping)
df

,item,user,rating,timestamp,type_id,user_id
1.0,A39HTATAQ9V7YF,0205616461,5.0,1369699200,2,0
2.0,A3JM6GV9MNOF9X,0558925278,3.0,1355443200,3,1
3.0,A1Z513UWSAAO0F,0558925278,5.0,1404691200,4,1
4.0,A1WMRR494NWEWV,0733001998,4.0,1382572800,5,2
5.0,A3IAAVS479H7M7,0737104473,1.0,1274227200,6,3
...,...,...,...,...,...,...
2023066.0,A3DEHKPFANB8VA,B00LORWRJA,5.0,1405296000,1207979,249269
2023067.0,A3DEHKPFANB8VA,B00LOS7MEE,5.0,1405296000,1207979,249270
2023068.0,AG9TJLJUN5OM3,B00LP2YB8E,5.0,1405382400,1210244,249271
2023069.0,AYBIB14QOI9PC,B00LPVG6V0,5.0,1405555200,1209898,249272


In [142]:
beauty,_ = make_to_sequence('user','item',f'{path}/beauty/ratings_Beauty.csv',1)
beauty

/var/folders/yn/ptj4__w518v924c18l9flhhw0000gn/T/ipykernel_1713/3295782531.py:2: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


,user_id,sequence_numeric
0,0,[2]
1,1,[3]
2,2,"[4, 5]"
3,3,[6]
4,4,[7]
...,...,...
249270,249270,[1207980]
249271,249271,[1207980]
249272,249272,[1210245]
249273,249273,[1209899]


In [145]:
beauty['sequence_length'] = beauty['sequence_numeric'].apply(len)
beauty['sequence_length'].describe()

count    249275.000000
mean          8.115820
std          36.064044
min           1.000000
25%           1.000000
50%           2.000000
75%           5.000000
max        7533.000000
Name: sequence_length, dtype: float64

In [150]:
beauty_ = beauty[(beauty['sequence_length'] >= 5) & (beauty['sequence_length'] <= 100)]
# beauty_ = beauty[beauty['sequence_length'] >= 5]
beauty_

,user_id,sequence_numeric,sequence_length
20,20,"[19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3...",16
23,23,"[37, 38, 35, 39, 40, 10]",6
51,51,"[63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73]",11
52,52,"[74, 75, 76, 77, 78, 79]",6
63,63,"[97, 98, 99, 100, 101, 102, 103, 104, 105, 106...",25
...,...,...,...
249248,249248,"[1210239, 1210240, 1209978, 1210241, 1210242]",5
249251,249251,"[1142422, 395725, 1177120, 1084709, 1159603, 1...",11
249262,249262,"[507822, 1158377, 1210251, 1149023, 1208738, 4...",10
249265,249265,"[1210254, 681319, 1210255, 1210256, 6237, 1210...",27


In [151]:
beauty_['sequence_length'].describe()

count    64658.000000
mean        17.040923
std         17.233669
min          5.000000
25%          6.000000
50%         10.000000
75%         20.000000
max        100.000000
Name: sequence_length, dtype: float64

In [120]:
unique_user_ids_beauty_ = beauty_['user_id'].unique()
filtered_beauty = df[df['user_id'].isin(unique_user_ids_beauty_)]
filtered_beauty

,item,user,rating,timestamp,type_id,user_id
5,A3IAAVS479H7M7,0737104473,1.0,1274227200,7,4
6,AKJHHD5VEH7VG,0762451459,5.0,1404518400,8,5
7,A1BG8QW55XHN6U,1304139212,5.0,1371945600,9,6
8,A22VW0P4VZHDE3,1304139220,5.0,1373068800,10,7
14,A22VW0P4VZHDE3,130414674X,5.0,1371686400,10,11
...,...,...,...,...,...,...
2022918,A2YLS6T097VC61,B00LAOMUUS,5.0,1405641600,1200618,249237
2022919,ADSYCM6P9HP0O,B00LAOMUUS,4.0,1405641600,1210221,249237
2022920,A1SOXC3E099RSJ,B00LAOMUUS,5.0,1405555200,1206456,249237
2022971,A1D7OVMZI4E4HT,B00LDNHQN2,4.0,1404691200,4318,249245


In [122]:
filtered_beauty['type_id'].nunique()

215808

In [105]:
filtered_beauty['item'].nunique()

567779

Amazon Books

In [ ]:
column_names = ['item', 'user','rating','timestamp']

df = pd.read_csv(f'{path}/books/books.csv', sep=',',header=None, names=column_names, engine= 'python')
df.to_csv(f'{path}/books/books.csv')